<a href="https://colab.research.google.com/github/dejiandrew/nba-award-predictor/blob/deji/pow_features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install wget
import pandas as pd
import numpy as np
import duckdb
import wget
import os

# Download CSV files
wget.download('https://storage.googleapis.com/nba_award_predictor/nba_data/nba-all-stars.csv')
wget.download('https://storage.googleapis.com/nba_award_predictor/nba_data/nba-mvp.csv')
wget.download('https://storage.googleapis.com/nba_award_predictor/nba_data/all-nba-first-team.csv')
wget.download('https://storage.googleapis.com/nba_award_predictor/nba_data/all-nba-second-team.csv')
wget.download('https://storage.googleapis.com/nba_award_predictor/nba_data/all-nba-third-team.csv')
wget.download('https://storage.googleapis.com/nba_award_predictor/nba_data/player-of-the-week.csv')
wget.download('https://storage.googleapis.com/nba_award_predictor/nba_data/player-statistics.csv')
wget.download('https://storage.googleapis.com/nba_award_predictor/nba_data/games.csv')

'games (1).csv'

In [2]:
games_df = pd.read_csv('games.csv')
games_df

/tmp/ipython-input-3918661511.py:1: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  games_df = pd.read_csv('games.csv')


,gameId,gameDate,hometeamCity,hometeamName,hometeamId,awayteamCity,awayteamName,awayteamId,homeScore,awayScore,winner,gameType,attendance,arenaId,gameLabel,gameSubLabel,seriesGameNumber
0,22500239,2025-11-16T20:00:00Z,Phoenix,Suns,1610612756,Atlanta,Hawks,1610612737,122,124,1610612737,NaN,17071.0,NaN,NaN,NaN,NaN
1,22500240,2025-11-16T20:00:00Z,Utah,Jazz,1610612762,Chicago,Bulls,1610612741,150,147,1610612762,NaN,18186.0,NaN,NaN,NaN,NaN
2,22500238,2025-11-16T19:30:00Z,Dallas,Mavericks,1610612742,Portland,Trail Blazers,1610612757,138,133,1610612742,NaN,18808.0,NaN,NaN,NaN,NaN
3,22500236,2025-11-16T19:00:00Z,Houston,Rockets,1610612745,Orlando,Magic,1610612753,117,113,1610612745,NaN,18055.0,NaN,NaN,NaN,NaN
4,22500237,2025-11-16T19:00:00Z,New Orleans,Pelicans,1610612740,Golden State,Warriors,1610612744,106,124,1610612744,NaN,18373.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72144,24600083,1946-12-08 19:00:00,New York,Knicks,1610612752,Boston,Celtics,1610612738,62,44,1610612752,Regular Season,NaN,0.0,NaN,NaN,NaN
72145,24600078,1946-12-07 19:00:00,Boston,Celtics,1610612738,New York,Knicks,1610612752,65,90,1610612752,Regular Season,NaN,0.0,NaN,NaN,NaN
72146,24600076,1946-12-05 19:00:00,Philadelphia,Warriors,1610612744,New York,Knicks,1610612752,62,51,1610612744,Regular Season,NaN,0.0,NaN,NaN,NaN
72147,24600063,1946-11-30 19:00:00,New York,Knicks,1610612752,Philadelphia,Warriors,1610612744,64,60,1610612752,Regular Season,NaN,0.0,NaN,NaN,NaN


In [3]:
def build_team_games(df, filter=None):
    df = df.copy()

    # Parse datetime with mixed formats
    df['gamedate'] = pd.to_datetime(
        df['gamedate'],
        utc=True,
        errors='coerce',
        format='mixed',
        dayfirst=True,
    )

    if filter:
        df = df.query(filter)

    # Normalize: make two rows per game (home + away)
    home = df.rename(columns={
        'hometeamname': 'team',
        'hometeamid':   'teamid',
        'homescore':    'team_score',
        'awayscore':    'opp_score'
    })[['gameid','gamedate','team','teamid','team_score','opp_score','winner',
        'awayteamname','awayteamid']].assign(
            home = 1,
            opponent  = lambda x: x['awayteamname'],
            opponentid= lambda x: x['awayteamid'],
        ).drop(columns=['awayteamname','awayteamid'])

    away = df.rename(columns={
        'awayteamname': 'team',
        'awayteamid':   'teamid',
        'awayscore':    'team_score',
        'homescore':    'opp_score'
    })[['gameid','gamedate','team','teamid','team_score','opp_score','winner',
        'hometeamname','hometeamid']].assign(
            home = 0,
            opponent  = lambda x: x['hometeamname'],
            opponentid= lambda x: x['hometeamid'],
        ).drop(columns=['hometeamname','hometeamid'])

    long = pd.concat([home, away], ignore_index=True)

    # Outcome flags
    long['is_win']  = (long['teamid'] == long['winner']).astype(int)
    long['outcome'] = long['is_win'].map({1: 'win', 0: 'loss'})
    long['is_home_win'] = ((long['home'] == 1) & (long['is_win'] == 1)).astype(int)
    long['is_away_win'] = ((long['home'] == 0) & (long['is_win'] == 1)).astype(int)

    # Season (season starts in July)
    year = long['gamedate'].dt.year
    month = long['gamedate'].dt.month
    long['season'] = np.where(month >= 7, year, year - 1)

    # Sort for rolling calculations
    long = long.sort_values(['teamid', 'season', 'gamedate', 'gameid'], kind='mergesort')

    # Group by team + season for all season-based stats
    g = long.groupby(['teamid', 'season'], group_keys=False)

    # Games played prior (per season)
    long['games_prior'] = g.cumcount()

    # Wins / losses prior (per season)
    long['wins_prior'] = g['is_win'].transform(
        lambda s: s.shift().fillna(0).cumsum().astype(int)
    )
    long['losses_prior'] = long['games_prior'] - long['wins_prior']

    # Home / away games prior (per season)
    long['home_games_prior'] = g['home'].transform(
        lambda s: s.shift(fill_value=0).cumsum()
    )
    long['away_games_prior'] = long['games_prior'] - long['home_games_prior']

    # Home / away wins prior (per season)
    long['home_wins_prior'] = g['is_home_win'].transform(
        lambda s: s.shift().fillna(0).cumsum().astype(int)
    )
    long['away_wins_prior'] = g['is_away_win'].transform(
        lambda s: s.shift().fillna(0).cumsum().astype(int)
    )

    # Home / away losses prior (per season)
    long['home_losses_prior'] = long['home_games_prior'] - long['home_wins_prior']
    long['away_losses_prior'] = long['away_games_prior'] - long['away_wins_prior']

    # Win streaks prior (per season)
    def streak_prior(series):
        prior = series.shift().fillna(0).astype(int)
        # reset when we see a 0
        return prior.groupby((prior == 0).cumsum()).cumsum().astype(int)

    long['win_streak_prior'] = g['is_win'].transform(streak_prior)
    long['home_win_streak_prior'] = g['is_home_win'].transform(streak_prior)
    long['away_win_streak_prior'] = g['is_away_win'].transform(streak_prior)

    # Record strings (per season)
    long['record_prior'] = long['wins_prior'].astype(str) + '-' + long['losses_prior'].astype(str)
    long['home_record_prior'] = long['home_wins_prior'].astype(str) + '-' + long['home_losses_prior'].astype(str)
    long['away_record_prior'] = long['away_wins_prior'].astype(str) + '-' + long['away_losses_prior'].astype(str)

    # Opponent's prior record for same game (still season-based because wins_prior/losses_prior are)
    opp_prior = long[['gameid','teamid','wins_prior','losses_prior']].rename(
        columns={
            'teamid': 'opponentid',
            'wins_prior': 'opp_wins_prior',
            'losses_prior': 'opp_losses_prior'
        }
    )

    long = long.merge(opp_prior, on=['gameid','opponentid'], how='left')

    # Opponent winrate prior to the game
    opp_games_prior = long['opp_wins_prior'] + long['opp_losses_prior']
    long['opp_winrate_prior'] = long['opp_wins_prior'] / opp_games_prior.where(opp_games_prior > 0)

    # Flag for wins vs > .500 opponent (per game)
    long['is_win_vs_over_500'] = (
        (long['is_win'] == 1) &
        (long['opp_winrate_prior'] > 0.5)
    ).astype(int)

    # Per-team, per-season cumulative PRIOR wins vs > .500
    long['wins_vs_over_500_prior'] = (
        long.groupby(['teamid', 'season'])['is_win_vs_over_500']
            .transform(lambda s: s.shift().fillna(0).cumsum().astype(int))
    )

    # Week-based (Mon–Sun) record PRIOR, per-season
    iso = long['gamedate'].dt.isocalendar()
    long['week_year'] = iso['year']
    long['week_num']  = iso['week']

    gw = long.groupby(['teamid','season','week_year','week_num'], group_keys=False)

    long['week_games_prior'] = gw.cumcount()
    long['week_wins_prior'] = gw['is_win'].transform(
        lambda s: s.shift().fillna(0).cumsum().astype(int)
    )
    long['week_losses_prior'] = long['week_games_prior'] - long['week_wins_prior']

    long['week_record_prior'] = (
        long['week_wins_prior'].astype(str)
        + '-' +
        long['week_losses_prior'].astype(str)
    )

    # Final output
    out = long[['gameid','gamedate',
                'team','teamid',
                'opponent','opponentid',
                'outcome','home',
                'team_score','opp_score',
                'games_prior','wins_prior','losses_prior','record_prior',
                'home_games_prior','home_wins_prior','home_losses_prior','home_record_prior',
                'away_games_prior','away_wins_prior','away_losses_prior','away_record_prior',
                'win_streak_prior','home_win_streak_prior','away_win_streak_prior',
                'opp_wins_prior','opp_losses_prior','opp_winrate_prior',
                'is_win_vs_over_500','wins_vs_over_500_prior',
                'week_games_prior','week_wins_prior','week_losses_prior','week_record_prior',
                'season'
               ]]
    out.loc[:, 'gameid'] = out['gameid'].astype(int)
    out = out.rename(columns={'gameid': 'gameId'})

    return out.sort_values(['gamedate','gameId','home']).reset_index(drop=True)

In [4]:
games_df.columns = games_df.columns.str.lower()

In [5]:
game = build_team_games(games_df, "gamedate.dt.year >= 1979")
game

,gameId,gamedate,team,teamid,opponent,opponentid,outcome,home,team_score,opp_score,...,opp_wins_prior,opp_losses_prior,opp_winrate_prior,is_win_vs_over_500,wins_vs_over_500_prior,week_games_prior,week_wins_prior,week_losses_prior,week_record_prior,season
0,27800397,1979-01-01 20:00:00+00:00,Warriors,1610612744,SuperSonics,1610612760,win,0,110,97,...,0,0,NaN,0,0,0,0,0,0-0,1978
1,27800397,1979-01-01 20:00:00+00:00,SuperSonics,1610612760,Warriors,1610612744,loss,1,97,110,...,0,0,NaN,0,0,0,0,0,0-0,1978
2,27800398,1979-01-02 20:00:00+00:00,Nuggets,1610612743,Kings,1610612758,win,0,99,97,...,0,0,NaN,0,0,0,0,0,0-0,1978
3,27800398,1979-01-02 20:00:00+00:00,Kings,1610612758,Nuggets,1610612743,loss,1,97,99,...,0,0,NaN,0,0,0,0,0,0-0,1978
4,27800399,1979-01-02 20:00:00+00:00,Bulls,1610612741,Bullets,1610612764,loss,0,86,109,...,0,0,NaN,0,0,0,0,0,0-0,1978
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115513,22500238,2025-11-16 19:30:00+00:00,Mavericks,1610612742,Trail Blazers,1610612757,win,1,138,133,...,7,9,0.437500,0,2,3,0,3,0-3,2025
115514,22500239,2025-11-16 20:00:00+00:00,Hawks,1610612737,Suns,1610612756,win,0,124,122,...,11,6,0.647059,1,3,3,3,0,3-0,2025
115515,22500239,2025-11-16 20:00:00+00:00,Suns,1610612756,Hawks,1610612737,loss,1,122,124,...,10,7,0.588235,0,3,3,3,0,3-0,2025
115516,22500240,2025-11-16 20:00:00+00:00,Bulls,1610612741,Jazz,1610612762,loss,0,147,150,...,5,11,0.312500,0,3,2,0,2,0-2,2025


In [6]:
game.isnull().sum()

,0
gameId,0
gamedate,0
team,0
teamid,0
opponent,0
opponentid,0
outcome,0
home,0
team_score,0
opp_score,0


In [7]:
query = """
WITH CTE AS (
SELECT
season
,team
,MIN(CAST(gamedate AS DATE)) AS min_gamedate
,MAX(CAST(gamedate AS DATE)) AS max_gamedate
FROM game
WHERE opp_winrate_prior IS NULL
GROUP BY season, team
)

SELECT *
,MONTH(min_gamedate) AS min_gamedate_month
,MONTH(max_gamedate) AS max_gamedate_month
FROM CTE
WHERE 1=1
AND min_gamedate_month NOT IN (9,10,11)           --This removes Sept,Oct,Nov games (1st game of season)
AND (season != 2020 AND min_gamedate_month != 7)  --Remove 2020 NBA Bubble scrimmages
AND (season != 1999 AND min_gamedate_month != 1)  --Remove 1999 lockout
AND (season != 1999 AND min_gamedate_month != 2)  --Remove 1999 lockout
AND (season != 2011 AND min_gamedate_month != 12) --Remove 2011 lockout


"""

duckdb.query(query).df()

# Based on this analysis, the null columns in game["opp_winrate_prior"] represent games where
# the opponent had played zero regular season games at that point. We can set game["opp_winrate_prior"]
# to 0.500 for these rows.


,season,team,min_gamedate,max_gamedate,min_gamedate_month,max_gamedate_month


In [8]:
game["opp_winrate_prior"] = game["opp_winrate_prior"].fillna(0.500)

In [9]:
game

,gameId,gamedate,team,teamid,opponent,opponentid,outcome,home,team_score,opp_score,...,opp_wins_prior,opp_losses_prior,opp_winrate_prior,is_win_vs_over_500,wins_vs_over_500_prior,week_games_prior,week_wins_prior,week_losses_prior,week_record_prior,season
0,27800397,1979-01-01 20:00:00+00:00,Warriors,1610612744,SuperSonics,1610612760,win,0,110,97,...,0,0,0.500000,0,0,0,0,0,0-0,1978
1,27800397,1979-01-01 20:00:00+00:00,SuperSonics,1610612760,Warriors,1610612744,loss,1,97,110,...,0,0,0.500000,0,0,0,0,0,0-0,1978
2,27800398,1979-01-02 20:00:00+00:00,Nuggets,1610612743,Kings,1610612758,win,0,99,97,...,0,0,0.500000,0,0,0,0,0,0-0,1978
3,27800398,1979-01-02 20:00:00+00:00,Kings,1610612758,Nuggets,1610612743,loss,1,97,99,...,0,0,0.500000,0,0,0,0,0,0-0,1978
4,27800399,1979-01-02 20:00:00+00:00,Bulls,1610612741,Bullets,1610612764,loss,0,86,109,...,0,0,0.500000,0,0,0,0,0,0-0,1978
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115513,22500238,2025-11-16 19:30:00+00:00,Mavericks,1610612742,Trail Blazers,1610612757,win,1,138,133,...,7,9,0.437500,0,2,3,0,3,0-3,2025
115514,22500239,2025-11-16 20:00:00+00:00,Hawks,1610612737,Suns,1610612756,win,0,124,122,...,11,6,0.647059,1,3,3,3,0,3-0,2025
115515,22500239,2025-11-16 20:00:00+00:00,Suns,1610612756,Hawks,1610612737,loss,1,122,124,...,10,7,0.588235,0,3,3,3,0,3-0,2025
115516,22500240,2025-11-16 20:00:00+00:00,Bulls,1610612741,Jazz,1610612762,loss,0,147,150,...,5,11,0.312500,0,3,2,0,2,0-2,2025


In [10]:
game.columns

Index(['gameId', 'gamedate', 'team', 'teamid', 'opponent', 'opponentid',
       'outcome', 'home', 'team_score', 'opp_score', 'games_prior',
       'wins_prior', 'losses_prior', 'record_prior', 'home_games_prior',
       'home_wins_prior', 'home_losses_prior', 'home_record_prior',
       'away_games_prior', 'away_wins_prior', 'away_losses_prior',
       'away_record_prior', 'win_streak_prior', 'home_win_streak_prior',
       'away_win_streak_prior', 'opp_wins_prior', 'opp_losses_prior',
       'opp_winrate_prior', 'is_win_vs_over_500', 'wins_vs_over_500_prior',
       'week_games_prior', 'week_wins_prior', 'week_losses_prior',
       'week_record_prior', 'season'],
      dtype='object')

In [11]:
pow_df = pd.read_csv('player-of-the-week.csv')
pow_df.columns[pow_df.isna().any()]

Index(['conference'], dtype='object')

In [12]:
query = """
WITH CTE AS (
SELECT
player_id
,season
,player
,CASE
    -- Eastern Conference
    WHEN team = 'Boston Celtics' THEN 'East'
    WHEN team = 'Brooklyn Nets' THEN 'East'
    WHEN team = 'New York Knicks' THEN 'East'
    WHEN team = 'Philadelphia 76ers' THEN 'East'
    WHEN team = 'Philadelphia Sixers' THEN 'East'
    WHEN team = 'Toronto Raptors' THEN 'East'
    WHEN team = 'Chicago Bulls' THEN 'East'
    WHEN team = 'Cleveland Cavaliers' THEN 'East'
    WHEN team = 'Detroit Pistons' THEN 'East'
    WHEN team = 'Indiana Pacers' THEN 'East'
    WHEN team = 'Milwaukee Bucks' THEN 'East'
    WHEN team = 'Atlanta Hawks' THEN 'East'
    WHEN team = 'Charlotte Hornets' THEN 'East'
    WHEN team = 'Miami Heat' THEN 'East'
    WHEN team = 'Orlando Magic' THEN 'East'
    WHEN team = 'Washington Wizards' THEN 'East'
    WHEN team = 'Washington Bullets' THEN 'East'
    WHEN team = 'New Jersey Nets' THEN 'East'
    WHEN team = 'Charlotte Bobcats' THEN 'East'

    -- Western Conference
    WHEN team = 'Denver Nuggets' THEN 'West'
    WHEN team = 'Minnesota Timberwolves' THEN 'West'
    WHEN team = 'Oklahoma City Thunder' THEN 'West'
    WHEN team = 'Portland Trail Blazers' THEN 'West'
    WHEN team = 'Utah Jazz' THEN 'West'
    WHEN team = 'Golden State Warriors' THEN 'West'
    WHEN team = 'Los Angeles Clippers' THEN 'West'
    WHEN team = 'Los Angeles Lakers' THEN 'West'
    WHEN team = 'Phoenix Suns' THEN 'West'
    WHEN team = 'Sacramento Kings' THEN 'West'
    WHEN team = 'Dallas Mavericks' THEN 'West'
    WHEN team = 'Houston Rockets' THEN 'West'
    WHEN team = 'Memphis Grizzlies' THEN 'West'
    WHEN team = 'New Orleans Pelicans' THEN 'West'
    WHEN team = 'San Antonio Spurs' THEN 'West'
    WHEN team = 'Seattle SuperSonics' THEN 'West'
    WHEN team = 'San Diego Clippers' THEN 'West'
    WHEN team = 'Kansas City Kings' THEN 'West'
    WHEN team = 'New Orleans Hornets' THEN 'West'
END AS conference

,date
,DAY(CAST(date AS DATE)) AS day
,WEEK(CAST(date AS DATE)) AS week
,MONTH(CAST(date AS DATE)) AS month
,YEAR(CAST(date AS DATE)) AS year
,team
,pos
,height
,weight
,age
,"Pre-Draft Team"
,"Draft Yr"
,yos

FROM pow_df
)

SELECT * FROM CTE

"""

pow_df = duckdb.query(query).df()
pow_df

,player_id,season,player,conference,date,day,week,month,year,team,pos,height,weight,age,Pre-Draft Team,Draft Yr,yos
0,1630595,2025-2026,Cade Cunningham,East,2025-11-10,10,46,11,2025,Detroit Pistons,SF,6-6,220,24,Oklahoma State,2021,4
1,203999,2025-2026,Nikola Jokic,West,2025-11-10,10,46,11,2025,Denver Nuggets,C,6-11,284,31,KK Mega Bemax (Serbia),2014,10
2,1630178,2025-2026,Tyrese Maxey,East,2025-11-03,3,45,11,2025,Philadelphia Sixers,PG,6-2,200,25,Kentucky,2020,5
3,1628983,2025-2026,Shai Gilgeous-Alexander,West,2025-11-03,3,45,11,2025,Oklahoma City Thunder,PG,6-6,195,27,Kentucky,2018,7
4,203507,2025-2026,Giannis Antetokounmpo,East,2025-10-27,27,44,10,2025,Milwaukee Bucks,F,6-11,243,31,Filathlitikos Div II Greece (Greece),2013,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1561,76736,1979-1980,Phil Ford,West,1979-11-18,18,46,11,1979,Kansas City Kings,G,6-2,175,24,North Carolina,1978,1
1562,77142,1979-1980,Magic Johnson,West,1979-11-11,11,45,11,1979,Los Angeles Lakers,PG,6-9,255,20,Michigan State,1979,0
1563,77160,1979-1980,Marques Johnson,East,1979-11-04,4,44,11,1979,Milwaukee Bucks,GF,6-7,218,24,UCLA,1977,2
1564,77952,1979-1980,Micheal Ray Richardson,East,1979-10-28,28,43,10,1979,New York Knicks,PG,6-5,189,24,Montana,1978,1


In [13]:
game.columns

Index(['gameId', 'gamedate', 'team', 'teamid', 'opponent', 'opponentid',
       'outcome', 'home', 'team_score', 'opp_score', 'games_prior',
       'wins_prior', 'losses_prior', 'record_prior', 'home_games_prior',
       'home_wins_prior', 'home_losses_prior', 'home_record_prior',
       'away_games_prior', 'away_wins_prior', 'away_losses_prior',
       'away_record_prior', 'win_streak_prior', 'home_win_streak_prior',
       'away_win_streak_prior', 'opp_wins_prior', 'opp_losses_prior',
       'opp_winrate_prior', 'is_win_vs_over_500', 'wins_vs_over_500_prior',
       'week_games_prior', 'week_wins_prior', 'week_losses_prior',
       'week_record_prior', 'season'],
      dtype='object')

In [14]:
query = """
SELECT
gameId
,CAST(gamedate AS DATE) AS gamedate
,DAY(CAST(gamedate AS DATE)) AS day
,WEEK(CAST(gamedate AS DATE)) AS week
,MONTH(CAST(gamedate AS DATE)) AS month
,YEAR(CAST(gamedate AS DATE)) AS year
,team
,teamid
,opponent
,opponentid
,outcome
,home
,team_score
,opp_score
,games_prior
,wins_prior
,losses_prior
,record_prior
,home_games_prior
,home_wins_prior
,home_losses_prior
,home_record_prior
,away_games_prior
,away_wins_prior
,away_losses_prior
,away_record_prior
,win_streak_prior
,home_win_streak_prior
,away_win_streak_prior
,opp_wins_prior
,opp_losses_prior
,opp_winrate_prior
,is_win_vs_over_500
,wins_vs_over_500_prior
,week_games_prior
,week_wins_prior
,week_losses_prior
,week_record_prior
,season

FROM game

"""

game = duckdb.query(query).df()
game

,gameId,gamedate,day,week,month,year,team,teamid,opponent,opponentid,...,opp_wins_prior,opp_losses_prior,opp_winrate_prior,is_win_vs_over_500,wins_vs_over_500_prior,week_games_prior,week_wins_prior,week_losses_prior,week_record_prior,season
0,27800397,1979-01-01,1,1,1,1979,Warriors,1610612744,SuperSonics,1610612760,...,0,0,0.500000,0,0,0,0,0,0-0,1978
1,27800397,1979-01-01,1,1,1,1979,SuperSonics,1610612760,Warriors,1610612744,...,0,0,0.500000,0,0,0,0,0,0-0,1978
2,27800398,1979-01-02,2,1,1,1979,Nuggets,1610612743,Kings,1610612758,...,0,0,0.500000,0,0,0,0,0,0-0,1978
3,27800398,1979-01-02,2,1,1,1979,Kings,1610612758,Nuggets,1610612743,...,0,0,0.500000,0,0,0,0,0,0-0,1978
4,27800399,1979-01-02,2,1,1,1979,Bulls,1610612741,Bullets,1610612764,...,0,0,0.500000,0,0,0,0,0,0-0,1978
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115513,22500238,2025-11-16,16,46,11,2025,Mavericks,1610612742,Trail Blazers,1610612757,...,7,9,0.437500,0,2,3,0,3,0-3,2025
115514,22500239,2025-11-16,16,46,11,2025,Hawks,1610612737,Suns,1610612756,...,11,6,0.647059,1,3,3,3,0,3-0,2025
115515,22500239,2025-11-16,16,46,11,2025,Suns,1610612756,Hawks,1610612737,...,10,7,0.588235,0,3,3,3,0,3-0,2025
115516,22500240,2025-11-16,16,46,11,2025,Bulls,1610612741,Jazz,1610612762,...,5,11,0.312500,0,3,2,0,2,0-2,2025


In [15]:
query = """
SELECT *
FROM game
JOIN pow_df
ON game.month = pow_df.month AND game.week = pow_df.week AND game.year = pow_df.year

"""

combined_df = duckdb.query(query).df()
combined_df = combined_df[['gameId', 'gamedate', 'team', 'teamid', 'opponent', 'opponentid',
       'outcome', 'home', 'team_score', 'opp_score', 'games_prior',
       'wins_prior', 'losses_prior', 'record_prior', 'home_games_prior',
       'home_wins_prior', 'home_losses_prior', 'home_record_prior',
       'away_games_prior', 'away_wins_prior', 'away_losses_prior',
       'away_record_prior', 'win_streak_prior', 'home_win_streak_prior',
       'away_win_streak_prior', 'opp_wins_prior', 'opp_losses_prior',
       'opp_winrate_prior', 'is_win_vs_over_500', 'wins_vs_over_500_prior',
       'week_games_prior', 'week_wins_prior', 'week_losses_prior',
       'week_record_prior', 'season', 'date', 'year', 'month', 'week', 'day',
       'conference', 'player', 'player_id', 'age', 'height', 'pos', 'yos']]
combined_df["won_player_of_the_week"] = 1
combined_df

,gameId,gamedate,team,teamid,opponent,opponentid,outcome,home,team_score,opp_score,...,week,day,conference,player,player_id,age,height,pos,yos,won_player_of_the_week
0,22000447,2021-02-18,Heat,1610612748,Kings,1610612758,win,0,118,110,...,7,18,West,Devin Booker,1626164,24,6-5,SG,5,1
1,22000447,2021-02-18,Kings,1610612758,Heat,1610612748,loss,1,110,118,...,7,18,West,Devin Booker,1626164,24,6-5,SG,5,1
2,22000449,2021-02-19,Warriors,1610612744,Magic,1610612753,loss,0,120,124,...,7,19,West,Devin Booker,1626164,24,6-5,SG,5,1
3,22000449,2021-02-19,Magic,1610612753,Warriors,1610612744,win,1,124,120,...,7,19,West,Devin Booker,1626164,24,6-5,SG,5,1
4,22000563,2021-02-19,Nuggets,1610612743,Cavaliers,1610612739,win,0,120,103,...,7,19,West,Devin Booker,1626164,24,6-5,SG,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135087,22000444,2021-02-17,Clippers,1610612746,Jazz,1610612762,loss,1,96,114,...,7,17,East,Saddiq Bey,1630180,21,6-8,F,0,1
135088,22000445,2021-02-18,Raptors,1610612761,Bucks,1610612749,win,0,110,96,...,7,18,East,Saddiq Bey,1630180,21,6-8,F,0,1
135089,22000445,2021-02-18,Bucks,1610612749,Raptors,1610612761,loss,1,96,110,...,7,18,East,Saddiq Bey,1630180,21,6-8,F,0,1
135090,22000446,2021-02-18,Nets,1610612751,Lakers,1610612747,win,0,109,98,...,7,18,East,Saddiq Bey,1630180,21,6-8,F,0,1


In [16]:
game.columns

Index(['gameId', 'gamedate', 'day', 'week', 'month', 'year', 'team', 'teamid',
       'opponent', 'opponentid', 'outcome', 'home', 'team_score', 'opp_score',
       'games_prior', 'wins_prior', 'losses_prior', 'record_prior',
       'home_games_prior', 'home_wins_prior', 'home_losses_prior',
       'home_record_prior', 'away_games_prior', 'away_wins_prior',
       'away_losses_prior', 'away_record_prior', 'win_streak_prior',
       'home_win_streak_prior', 'away_win_streak_prior', 'opp_wins_prior',
       'opp_losses_prior', 'opp_winrate_prior', 'is_win_vs_over_500',
       'wins_vs_over_500_prior', 'week_games_prior', 'week_wins_prior',
       'week_losses_prior', 'week_record_prior', 'season'],
      dtype='object')

In [17]:
player_statistics_df = pd.read_csv('player-statistics.csv')
player_statistics_df.head()

/tmp/ipython-input-1148877721.py:1: DtypeWarning: Columns (11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  player_statistics_df = pd.read_csv('player-statistics.csv')


,firstName,lastName,full_name,player_id,gameId,gameDate,playerteamCity,playerteamName,opponentteamCity,opponentteamName,...,threePointersPercentage,freeThrowsAttempted,freeThrowsMade,freeThrowsPercentage,reboundsDefensive,reboundsOffensive,reboundsTotal,foulsPersonal,turnovers,plusMinusPoints
0,Devin,Booker,Devin Booker,1626164,22500239,2025-11-16T20:00:00Z,Phoenix,Suns,Atlanta,Hawks,...,0.000,7.0,7.0,1.000,1.0,1.0,2.0,4.0,5.0,-17.0
1,Royce,O'Neale,Royce O'Neale,1626220,22500239,2025-11-16T20:00:00Z,Phoenix,Suns,Atlanta,Hawks,...,0.167,0.0,0.0,0.000,4.0,2.0,6.0,1.0,0.0,-18.0
2,Lauri,Markkanen,Lauri Markkanen,1628374,22500240,2025-11-16T20:00:00Z,Utah,Jazz,Chicago,Bulls,...,0.353,13.0,11.0,0.846,5.0,2.0,7.0,3.0,1.0,4.0
3,Luke,Kennard,Luke Kennard,1628379,22500239,2025-11-16T20:00:00Z,Atlanta,Hawks,Phoenix,Suns,...,0.000,0.0,0.0,0.000,1.0,0.0,1.0,0.0,0.0,-4.0
4,Dillon,Brooks,Dillon Brooks,1628415,22500239,2025-11-16T20:00:00Z,Phoenix,Suns,Atlanta,Hawks,...,0.429,3.0,3.0,1.000,5.0,1.0,6.0,5.0,3.0,-2.0


In [18]:
player_statistics_df.columns
# player_statistics_df.merge(combined_df,on='player_id',how='inner')

Index(['firstName', 'lastName', 'full_name', 'player_id', 'gameId', 'gameDate',
       'playerteamCity', 'playerteamName', 'opponentteamCity',
       'opponentteamName', 'gameType', 'gameLabel', 'gameSubLabel',
       'seriesGameNumber', 'win', 'home', 'numMinutes', 'points', 'assists',
       'blocks', 'steals', 'fieldGoalsAttempted', 'fieldGoalsMade',
       'fieldGoalsPercentage', 'threePointersAttempted', 'threePointersMade',
       'threePointersPercentage', 'freeThrowsAttempted', 'freeThrowsMade',
       'freeThrowsPercentage', 'reboundsDefensive', 'reboundsOffensive',
       'reboundsTotal', 'foulsPersonal', 'turnovers', 'plusMinusPoints'],
      dtype='object')

In [19]:
player_statistics_df = player_statistics_df[['firstName', 'lastName', 'full_name', 'player_id', 'gameId', 'gameDate',
  'numMinutes', 'points', 'assists',
       'blocks', 'steals', 'fieldGoalsAttempted', 'fieldGoalsMade',
       'fieldGoalsPercentage', 'threePointersAttempted', 'threePointersMade',
       'threePointersPercentage', 'freeThrowsAttempted', 'freeThrowsMade',
       'freeThrowsPercentage', 'reboundsDefensive', 'reboundsOffensive',
       'reboundsTotal', 'foulsPersonal', 'turnovers', 'plusMinusPoints']]
player_statistics_df

,firstName,lastName,full_name,player_id,gameId,gameDate,numMinutes,points,assists,blocks,...,threePointersPercentage,freeThrowsAttempted,freeThrowsMade,freeThrowsPercentage,reboundsDefensive,reboundsOffensive,reboundsTotal,foulsPersonal,turnovers,plusMinusPoints
0,Devin,Booker,Devin Booker,1626164,22500239,2025-11-16T20:00:00Z,37.36,27.0,7.0,1.0,...,0.000,7.0,7.0,1.000,1.0,1.0,2.0,4.0,5.0,-17.0
1,Royce,O'Neale,Royce O'Neale,1626220,22500239,2025-11-16T20:00:00Z,24.56,3.0,1.0,0.0,...,0.167,0.0,0.0,0.000,4.0,2.0,6.0,1.0,0.0,-18.0
2,Lauri,Markkanen,Lauri Markkanen,1628374,22500240,2025-11-16T20:00:00Z,46.22,47.0,2.0,2.0,...,0.353,13.0,11.0,0.846,5.0,2.0,7.0,3.0,1.0,4.0
3,Luke,Kennard,Luke Kennard,1628379,22500239,2025-11-16T20:00:00Z,11.35,0.0,0.0,0.0,...,0.000,0.0,0.0,0.000,1.0,0.0,1.0,0.0,0.0,-4.0
4,Dillon,Brooks,Dillon Brooks,1628415,22500239,2025-11-16T20:00:00Z,35.32,34.0,1.0,0.0,...,0.429,3.0,3.0,1.000,5.0,1.0,6.0,5.0,3.0,-2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1635253,Fred,Sheffield,Fred Sheffield,78131,24600052,1946-11-26 19:00:00,0.00,1.0,0.0,0.0,...,0.000,0.0,1.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0
1635254,Connie,Simmons,Connie Simmons,78153,24600052,1946-11-26 19:00:00,0.00,15.0,0.0,0.0,...,0.000,0.0,5.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0
1635255,Johnny,Simmons,Johnny Simmons,78154,24600052,1946-11-26 19:00:00,0.00,14.0,0.0,0.0,...,0.000,0.0,2.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0
1635256,Virgil,Vaughn,Virgil Vaughn,78411,24600052,1946-11-26 19:00:00,0.00,4.0,0.0,0.0,...,0.000,0.0,2.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
query = """

SELECT
player_statistics_df.firstName
,player_statistics_df.lastName
,player_statistics_df.full_name
,player_statistics_df.player_id
,player_statistics_df.gameId
,CAST(player_statistics_df.gameDate AS DATE) AS gameDate
,DAY(CAST(player_statistics_df.gameDate AS DATE)) AS day
,MONTH(CAST(player_statistics_df.gameDate AS DATE)) AS month
,YEAR(CAST(player_statistics_df.gameDate AS DATE)) AS year
,numMinutes
,points
,assists
,blocks
,steals
,fieldGoalsAttempted
,fieldGoalsMade
,fieldGoalsPercentage
,threePointersAttempted
,threePointersMade
,threePointersPercentage
,freeThrowsAttempted
,freeThrowsMade
,freeThrowsPercentage
,reboundsDefensive
,reboundsOffensive
,reboundsTotal
,foulsPersonal
,turnovers
,plusMinusPoints
,combined_df.*

FROM player_statistics_df
LEFT JOIN combined_df
ON player_statistics_df.player_id = combined_df.player_id AND player_statistics_df.gameId = combined_df.gameId
WHERE 1=1
AND numMinutes IS NOT NULL  -- Remove any games where a player didn't play

"""

player_stats_with_pow = duckdb.query(query).df()
player_stats_with_pow["won_player_of_the_week"] = np.where(player_stats_with_pow["won_player_of_the_week"].isna(),0,1)
player_stats_with_pow[[
    'firstName', 'lastName', 'full_name', 'player_id', 'gameId', 'gameDate',
    'numMinutes', 'points', 'assists',
    'blocks', 'steals', 'fieldGoalsAttempted', 'fieldGoalsMade',
    'fieldGoalsPercentage', 'threePointersAttempted', 'threePointersMade',
    'threePointersPercentage', 'freeThrowsAttempted', 'freeThrowsMade',
    'freeThrowsPercentage', 'reboundsDefensive', 'reboundsOffensive',
    'reboundsTotal', 'foulsPersonal', 'turnovers', 'plusMinusPoints', 'won_player_of_the_week'
]]

,firstName,lastName,full_name,player_id,gameId,gameDate,numMinutes,points,assists,blocks,...,freeThrowsAttempted,freeThrowsMade,freeThrowsPercentage,reboundsDefensive,reboundsOffensive,reboundsTotal,foulsPersonal,turnovers,plusMinusPoints,won_player_of_the_week
0,Elfrid,Payton,Elfrid Payton,203901,21600730,2017-02-01,31.0,11.0,3.0,0.0,...,1.0,1.0,1.000,6.0,0.0,6.0,3.0,1.0,-6.0,0
1,Glenn,Robinson III,Glenn Robinson III,203922,21600730,2017-02-01,21.0,14.0,0.0,0.0,...,2.0,2.0,1.000,6.0,1.0,7.0,1.0,0.0,6.0,0
2,Aaron,Gordon,Aaron Gordon,203932,21600730,2017-02-01,31.0,7.0,1.0,1.0,...,3.0,1.0,0.333,0.0,0.0,0.0,1.0,1.0,-2.0,0
3,Adreian,Payne,Adreian Payne,203940,21600731,2017-02-01,3.0,0.0,0.0,0.0,...,0.0,0.0,0.000,1.0,0.0,1.0,1.0,0.0,-2.0,0
4,Andrew,Wiggins,Andrew Wiggins,203952,21600731,2017-02-01,35.0,23.0,0.0,0.0,...,3.0,3.0,1.000,1.0,1.0,2.0,2.0,2.0,-14.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1476968,Reggie,Miller,Reggie Miller,397,20401084,2005-04-03,34.0,12.0,3.0,0.0,...,4.0,4.0,1.000,4.0,0.0,4.0,1.0,1.0,-3.0,1
1476969,Baron,Davis,Baron Davis,1884,20401075,2005-04-01,29.0,12.0,8.0,1.0,...,5.0,3.0,0.600,4.0,3.0,7.0,3.0,3.0,7.0,1
1476970,Ray,Allen,Ray Allen,951,20401036,2005-03-27,43.0,27.0,2.0,0.0,...,12.0,11.0,0.917,7.0,1.0,8.0,2.0,3.0,12.0,1
1476971,Allen,Iverson,Allen Iverson,947,20401035,2005-03-27,42.0,20.0,15.0,0.0,...,14.0,10.0,0.714,1.0,1.0,2.0,2.0,4.0,3.0,1
